# BRIDGE CROSSING PROBLEM

October 12, 2017

Welcome! This IPython notebook will go through the implementation of the the Uniform Cost Search Algorithm and A\* Search Algorithm to solve the famous Bridge Crossing Problem. An analysis and comparison of the two algorithms will also be shown.

# 1. Search Algorithms

RYAN...TASK ENVIRONMENT = (Performance Measure, External Environment, Actuators, Sensors)

The state of the environment is represented as a list of boolean values with the *nth* element of the list corresponding to the *nth* person on the environment. The value of 0 means that the *nth* person has not crossed the bridge (at the left of the bridge), while the value of 1 means that the *nth* person crossed the bridge (at the right of the bridge). The state $[0,0,0,...,0]$ is an initial state, while $[1,1,1,...,1]$ is the goal state. An example is shown below.

*e.g.* Given 4 people, the state $[1,0,0,1]$ means that the *1st* and *4th* person already crossed, and the *2nd* and *3rd* hasn't.

The state space, on the other hand, is represented as a tree. The tree has multiple nodes and multiple branches. Nodes contain useful information about the current state of the environment, while the branches connect two nodes and it has a cost corresponding to the cost of going from the initial state to the current state.

## 1.1 Uniform Cost Search

Useful libraries are imported. Bisect is used in this project for optimally inserting elements in a list in a sorted manner, while Itertools is used to generate a combination of a list.

In [2]:
import bisect, itertools

To reduce code complexity, the nodes of a tree are objects with cost, state, and steps as attributes.

*COST (Integer) -->* cost of going from the initial state to current state.<br />
*STATE (List) -->* the current state of the environment as defined above.<br />
*STEPS (List) -->* the sequence of steps of the people leading to the current state.<br />

*e.g.* Given a Node $(10, [1,0,0,0], [1,3,3])$ means that the cost from initial state to current state, $[1,0,0,0]$, is 10. Moreover, the sequence of steps from initial state to current state is $[1,3,3]$, which means that 1 and 3 has crossed the bridge and 3 went back with the flashlight.

Notice that the inclusion of the steps attribute will aid with the identification of the path from the initial state to the goal state. With this, the sequence of steps of the solution can easily be printed out. The function print_steps uses the steps attribute to print the sequence of steps.

In [5]:
class Node:
    def __init__(self, cost, state, steps):
        self.cost = cost
        self.state = state
        self.steps = steps

    def __lt__(self, other):
        return self.cost < other.cost

    def print_steps(self):
        i = 0
        while i < len(self.steps):
            if (i+1) % 3 == 0:
                print(self.steps[i], '<-')
                i += 1
            else:
                print(self.steps[i], self.steps[i+1], '->')
                i += 2

<br />The code snippet below is the Tree class. This is the code representation of the state space. The class contains the following attributes:

*FRINGE (List) -->* collection of nodes yet to be expanded.<br />
*FITNESS (List) -->* the original input; a list of the time required for the people to cross the bridge.

The fringe initially contains the root node, which contains the initial state, a cost of 0 (since it is the root node), and an empty steps list. The different class methods will be explained after the code snippet below.

In [9]:
class Tree:
    def __init__(self, fitness):
        state = [0] * len(fitness)
        steps = []
        cost = 0
        root = Node(cost, state, steps)

        self.fringe = [root]
        self.fitness = fitness

    def has_repeated_states(self, node):
        for item in self.fringe:
            if node.state == item.state and node.cost < item.cost:
                self.fringe.remove(item)
                bisect.insort(self.fringe, node)
                return True
            if node.state == item.state:
                return True
        return False

    def generate_nodes(self, node):
        fitness_len = len(self.fitness)
        if len(node.steps) % 3 == 0:
            for elem in itertools.combinations(list(range(fitness_len)), 2):
                if (node.state[elem[0]] == 0 and node.state[elem[1]] == 0):
                    new_state = list(node.state)
                    new_state[elem[0]] = 1
                    new_state[elem[1]] = 1

                    new_steps = list(node.steps)
                    new_steps.extend([elem[0]+1, elem[1]+1])

                    new_cost = node.cost + max(self.fitness[elem[0]], self.fitness[elem[1]])

                    new_node = Node(new_cost, new_state, new_steps)

                    # Check for Repeated States
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

        else:
            for elem in list(range(fitness_len)):
                if (node.state[elem] == 1):
                    new_state = list(node.state)
                    new_state[elem] = 0

                    new_steps = list(node.steps)
                    new_steps.append(elem+1)

                    new_cost = node.cost + self.fitness[elem]

                    new_node = Node(new_cost, new_state, new_steps)

                    # Check for Repeated States
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

    def uniform_cost_search(self):
        num_visited = 0
        fitness_len = len(self.fitness)
        while True:
            if not self.fringe:
                return 'Failure'

            # Explore Fringe
            node = self.fringe.pop(0)
            num_visited += 1

            # Check for Goal State
            if node.state == [1] * fitness_len:
                print(node.cost, num_visited)
                node.print_steps()
                return 'Success'

            # Generate Successor States
            self.generate_nodes(node)

<br />This class method contains the Uniform Cost Search algorithm. It continues to loop until either a goal state is found or if there exists no goal state. If the fringe is empty, then there exists no goal state. The algorithm continues to pop a node from the fringe and adds the successor nodes of that node inside the fringe while keeping the fringe sorted. As a modification to the algorithm, a loop counter was placed to identify the number of expanded nodes.

The successor nodes are generated by calling the class method generate_nodes with an argument of the popped node.

In [6]:
def uniform_cost_search(self):
        num_visited = 0
        fitness_len = len(self.fitness)
        while True:
            if not self.fringe:
                return 'Failure'

            # Explore Fringe
            node = self.fringe.pop(0)
            num_visited += 1

            # Check for Goal State
            if node.state == [1] * fitness_len:
                print(node.cost, num_visited)
                node.print_steps()
                return 'Success'

            # Generate Successor States
            self.generate_nodes(node)

<br />Insert Description Here, Explain that this is part of the class Tree()

In [7]:
def generate_nodes(self, node):
        fitness_len = len(self.fitness)
        if len(node.steps) % 3 == 0:
            for elem in itertools.combinations(list(range(fitness_len)), 2):
                if (node.state[elem[0]] == 0 and node.state[elem[1]] == 0):
                    new_state = list(node.state)
                    new_state[elem[0]] = 1
                    new_state[elem[1]] = 1

                    new_steps = list(node.steps)
                    new_steps.extend([elem[0]+1, elem[1]+1])

                    new_cost = node.cost + max(self.fitness[elem[0]], self.fitness[elem[1]])

                    new_node = Node(new_cost, new_state, new_steps)

                    # Check for Repeated States
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

        else:
            for elem in list(range(fitness_len)):
                if (node.state[elem] == 1):
                    new_state = list(node.state)
                    new_state[elem] = 0

                    new_steps = list(node.steps)
                    new_steps.append(elem+1)

                    new_cost = node.cost + self.fitness[elem]

                    new_node = Node(new_cost, new_state, new_steps)

                    # Check for Repeated States
                    if not self.has_repeated_states(new_node):
                        bisect.insort(self.fringe, new_node)

<br />The has_repeated_states class method checks if the state of a node (node.state) is already contained within the fringe. This is a crucial part of the Uniform Cost Search algorithm since a repeated state is to be avoided. The If node.state is in the fringe, the path cost to the node has to be checked if it is less than the path cost of the node stored in the fringe. If it is, the node in the fringe has to be replaced. If it is not, nothing has to be done.

The has_repeated_states class method takes in a node as a parameter. The usual argument being passed into this method is a newly generated node from the generate_nodes method.

In [8]:
def has_repeated_states(self, node):
        for item in self.fringe:
            if node.state == item.state and node.cost < item.cost:
                self.fringe.remove(item)
                bisect.insort(self.fringe, node)
                return True
            if node.state == item.state:
                return True
        return False

Insert Description Here, Explain that this is THE class Tree()

## 1.2 A\* Search

Insert Description Here